# Udacity MLND Capstone — Tweet Classifier

## Prep
* load libraries
* establish baseline algorithm (Multinomial Naive Bayes)

In [1]:
import numpy as np
import pandas as pd

from keras.preprocessing import sequence

Using TensorFlow backend.


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

tweets = pd.read_csv("./ExtractedTweets.csv")

---

## Baseline Algorithm (Multinomial NB)

In [3]:

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(tweets.Tweet)

print(X_train_counts.shape)

(86460, 126330)


In [4]:

tweet_accounts = tweets.iloc[:, :2].drop_duplicates()
accounts_train, accounts_test = train_test_split(tweet_accounts.Handle, stratify=tweet_accounts.Party, \
                                                 test_size=0.2, random_state=41)

tweets_train = tweets[tweets.Handle.isin(accounts_train)].reset_index().drop('index', axis=1)
tweets_test = tweets[tweets.Handle.isin(accounts_test)].reset_index().drop('index', axis=1)

In [5]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

mnb_pipeline = Pipeline([('vec', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

mnb_pipeline.fit(tweets_train.Tweet, tweets_train.Party)

p = mnb_pipeline.predict(tweets_test.Tweet)
np.mean(p == tweets_test.Party)

0.75217089267106629

---

## Neural Networks

### Tensor Setup

In [6]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling1D, MaxPooling3D
from keras.layers.embeddings import Embedding

from nltk.tokenize.casual import TweetTokenizer
from nltk import FreqDist

tokenizer = TweetTokenizer(reduce_len=True)

top_words = 25000

tweets_nn = tweets
tweets_nn.Tweet = tweets_nn.Tweet.apply(tokenizer.tokenize)

In [7]:
print("...")

fdist = FreqDist(word for tweet in tweets_nn.Tweet for word in tweet)
print("...fdist")
terms = [term for term, count in fdist.most_common(top_words)]
print("...terms")
tweets_nn.Tweet = tweets_nn.Tweet.apply(\
                    lambda tweet:[terms.index(term)\
                    if term in terms else 0 for term in tweet]
                )
print("...term lambda")

nn_tweeters = tweets_nn.iloc[:,:2].drop_duplicates()

print("^^^")

...
...fdist
...terms
...term lambda
^^^


In [8]:

nn_tweeters_train, nn_tweeters_test = train_test_split(nn_tweeters.Handle,\
                           stratify=nn_tweeters.Party, test_size=0.2, random_state=42)

nn_tweets_train = tweets_nn[tweets_nn.Handle.isin(nn_tweeters_train)].reset_index().drop('index', axis=1)
nn_tweets_test = tweets_nn[tweets_nn.Handle.isin(nn_tweeters_test)].reset_index().drop('index',axis=1)


In [9]:
print(nn_tweets_train.shape)
print(nn_tweets_test.shape)
nn_tweets_test.head()

(69185, 3)
(17275, 3)


,Party,Handle,Tweet
0,Democrat,RepJackyRosen,"[383, 3, 24, 480, 15, 19, 0, 4, 0, 0, 20051, 1..."
1,Democrat,RepJackyRosen,"[25, 1610, 2124, 5496, 469, 4, 119, 108, 7, 71..."
2,Democrat,RepJackyRosen,"[168, 275, 3, 0, 6427, 10, 40, 0, 1673, 151, 0..."
3,Democrat,RepJackyRosen,"[3872, 5616, 3, 42, 0, 104, 41, 261, 24, 5683,..."
4,Democrat,RepJackyRosen,"[52, 22, 4, 4765, 4, 10, 1627, 80, 3, 24, 110,..."


In [10]:
nn_tweets_train.head()


,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"[51, 4, 222, 1206, 161, 3, 3266, 2, 191, 3, 77..."
1,Democrat,RepDarrenSoto,"[11, 21801, 6, 4360, 5949, 3405, 60, 0, 7132, ..."
2,Democrat,RepDarrenSoto,"[11, 12210, 6, 2, 1973, 10955, 21, 1217, 3012,..."
3,Democrat,RepDarrenSoto,"[11, 0, 6, 1343, 17, 1973, 2, 87, 10, 381, 0, ..."
4,Democrat,RepDarrenSoto,"[11, 0, 6, 1217, 1078, 1816, 13, 2555, 752, 45..."


In [11]:
np.random.seed(42)

X_train = np.array(nn_tweets_train.Tweet)
X_test = np.array(nn_tweets_test.Tweet)
y_train = np.array((nn_tweets_train.Party == 'Democrat').astype(int))
y_test = np.array((nn_tweets_test.Party == 'Democrat').astype(int))

In [12]:
tweet_length = 60
X_train = sequence.pad_sequences(X_train, maxlen=tweet_length)
X_test = sequence.pad_sequences(X_test, maxlen=tweet_length)

### Neural Net — Model 1 — LSTM Recurrent Neural Network

In [13]:
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=tweet_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
#model.add(Dropout(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
#model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 60, 32)            800000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 60, 32)            3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 30, 32)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 856,405
Trainable params: 856,405
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
69185/69185 [==============================] - 52s - loss: 0.4841 - acc: 0.7482    
Epoch 2/3
69185/69185 [==============

### Neural Net — Model 2 — VGG Net Basis

In [15]:

print(X_train.shape)

model2 = Sequential()

model2.add(Embedding(top_words, embedding_vector_length, input_length=tweet_length))

model2.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(Dropout(0.2))

model2.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(Dropout(0.2))

model2.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(Dropout(0.2))

model2.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(Dropout(0.2))

model2.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(Dropout(0.2))

model2.add(LSTM(100))

model2.add(Dense(1, activation='sigmoid'))

model2.summary()


(69185, 60)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 60, 32)            800000    
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 60, 64)            6208      
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 30, 64)            0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 30, 64)            0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 30, 64)            12352     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 15, 64)            0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 15, 64)            0        

In [16]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',])
model2.fit(X_train, y_train, epochs=4, batch_size=64)
scores2 = model2.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores2[1]*100))

Epoch 1/4
69185/69185 [==============================] - 13s - loss: 0.5228 - acc: 0.7055    
Epoch 2/4
69185/69185 [==============================] - 13s - loss: 0.3275 - acc: 0.8501    
Epoch 3/4
69185/69185 [==============================] - 13s - loss: 0.2455 - acc: 0.8910    
Epoch 4/4
69185/69185 [==============================] - 13s - loss: 0.1811 - acc: 0.9206    
Accuracy: 72.29%


### Neural Net 3 — Bigger VGG Net

In [17]:

embedding_vector_length_longer = 64
tweet_length = 80
X_train = sequence.pad_sequences(X_train, maxlen=tweet_length)
X_test = sequence.pad_sequences(X_test, maxlen=tweet_length)
print(X_train.shape)

model3 = Sequential()

model3.add(Embedding(top_words, embedding_vector_length_longer, input_length=tweet_length))

model3.add(Conv1D(filters=256, kernel_size=12, padding='same', activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.2))

model3.add(Conv1D(filters=256, kernel_size=12, padding='same', activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.2))

model3.add(Conv1D(filters=256, kernel_size=12, padding='same', activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.2))

model3.add(Conv1D(filters=256, kernel_size=12, padding='same', activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.2))

model3.add(Conv1D(filters=256, kernel_size=12, padding='same', activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.2))

#model3.add(Conv1D(filters=256, kernel_size=12, padding='same', activation='relu'))
#model3.add(MaxPooling1D(pool_size=2))
#model3.add(Dropout(0.2))


model3.add(LSTM(400, dropout=0.2, recurrent_dropout=0.2))
model3.add(Dense(1, activation='sigmoid'))

model3.summary()

(69185, 80)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 80, 64)            1600000   
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 80, 256)           196864    
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 40, 256)           0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 40, 256)           0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 40, 256)           786688    
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 20, 256)           0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 20, 256)           0        

In [18]:
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',])
model3.fit(X_train, y_train, epochs=4, batch_size=64)
scores3 = model3.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores3[1]*100))

Epoch 1/4
69185/69185 [==============================] - 50s - loss: 0.5024 - acc: 0.7268    
Epoch 2/4
69185/69185 [==============================] - 48s - loss: 0.3068 - acc: 0.8583    
Epoch 3/4
69185/69185 [==============================] - 48s - loss: 0.1936 - acc: 0.9161    
Epoch 4/4
69185/69185 [==============================] - 48s - loss: 0.1033 - acc: 0.9580    
Accuracy: 71.50%
